In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# Import data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Read training data from csv file.
train_data_path = '/kaggle/input/digit-recognizer/train.csv'
train_data = pd.read_csv(train_data_path)
train_data.head()

# Data exploration

In [ ]:
train_labels = train_data[['label']]
train_labels.head()
index = dict()

for label in train_labels.label.unique():
    index[label] = train_labels.loc[train_labels['label'] == label].index

In [ ]:
# Inspect the number of data in each class
count_of_each_label = train_labels.label.value_counts(sort=False)
print(count_of_each_label)
count_of_each_label.plot(kind="bar", rot=0);

In [ ]:
# create train and val set, 80-20 ratio for each class
from sklearn.model_selection import train_test_split

train_index = []
val_index = []
for label in train_labels.label.unique():
    train, val = train_test_split(index[label], train_size=0.8, shuffle=True, random_state=42)
    train_index = [*train_index, *train]
    val_index = [*val_index, *val]
    
train_X = train_data.iloc[train_index].drop(columns=['label'])
train_y = train_data.iloc[train_index][['label']]
val_X = train_data.iloc[val_index].drop(columns=['label'])
val_y = train_data.iloc[val_index][['label']]

In [ ]:
# Take all the value for reshaping -1, dimension (28,28), colour 1
train_X = train_X.values.reshape(-1,28,28)
val_X = val_X.values.reshape(-1,28,28)

In [ ]:
from matplotlib import pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

%matplotlib inline

plt.imshow(train_X[3], cmap='gray');

In [ ]:
# Transfer Learning requirement
# Convert to 3 channel images RGB
train_X = np.repeat(train_X[..., np.newaxis], 3, -1)
val_X = np.repeat(val_X[..., np.newaxis], 3, -1)
print(train_X.shape)
# train_X = np.expand_dims(train_X, axis=-1)
# val_X = np.expand_dims(val_X, axis=-1)

In [ ]:
# Resize into 32,32 images
train_X = tf.image.resize(train_X, [32, 32], method='nearest')
val_X = tf.image.resize(val_X, [32, 32], method='nearest')
print(train_X.shape)

# Data augmentation

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rotation_range=5, width_shift_range=0.1,height_shift_range=0.2, zoom_range=0.1)
train_datagen.fit(train_X, seed=42)
train_tranformed = train_datagen.flow(train_X, train_y, batch_size=32)

val_datagen = ImageDataGenerator()
val_datagen.fit(val_X, seed=42)
val_tranformed = val_datagen.flow(val_X, val_y, batch_size=32)

In [ ]:
train_X.shape

In [ ]:
train_tranformed.next()[0].shape

In [ ]:
fig, ax = plt.subplots(4, 4, figsize=(15, 15))
for row in range(4):
    for col in range(4):
        # generate batch of images
        batch = train_tranformed.next()
        # convert to unsigned integers for viewing
        image = batch[0].astype('uint8')
        # plot raw pixel data
        ax[row, col].imshow(np.squeeze(image[0]))

# Create layers

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import datetime

def create_model(image_shape):
    base_model = tf.keras.applications.VGG16(include_top=False, pooling='avg', input_shape=image_shape, weights='/kaggle/input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

    # Flatten the CNN
    base_output = base_model.output
    flatten = tf.keras.layers.Flatten()(base_output)

    # Dense layer
    dense1 = tf.keras.layers.Dense(128, activation='relu')(flatten)
    dense2 = tf.keras.layers.Dense(64, activation='relu')(dense1)
    output = tf.keras.layers.Dense(10, activation='softmax')(dense2)

    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    # Freeze all base layers
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.001),
        metrics=['accuracy'],
    )
    return model

save_model_name = datetime.datetime.now()
checkpoint1 = ModelCheckpoint("{}.h5".format(save_model_name), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint1]

input_shape = train_tranformed.next()[0].shape[1:]
model = create_model(input_shape)
model.summary()

# Training

In [ ]:
history = model.fit(x=train_tranformed, steps_per_epoch=len(train_X) / 32, 
                    validation_data=val_tranformed, validation_steps=len(val_X) / 32,
                    epochs=30, callbacks=callbacks_list)

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Evaluation

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('{}.h5'.format(save_model_name))

In [ ]:
# Read in the test features.
test_data_path = '/kaggle/input/digit-recognizer/test.csv'
test_data = pd.read_csv(test_data_path)
# test_data.head()
test_data = test_data.values.reshape(-1,28,28)
test_data = np.repeat(test_data[..., np.newaxis], 3, -1)
test_data = tf.image.resize(test_data, [32, 32], method='nearest')

In [ ]:
test_data.shape

In [ ]:
# Predict the all the test features.
test_probs = new_model.predict(test_data)
test_labels = test_probs.argmax(axis=-1)

In [ ]:
# Generate Submission File.
submission_df = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
submission_df.Label = test_labels
submission_df.to_csv("output.csv", index=False)

https://www.kaggle.com/c/digit-recognizer